In [1]:
import pandas as pd
import requests
from bs4 import BeautifulSoup

In [2]:
url = "https://www.caffeineinformer.com/the-caffeine-database/"
response = requests.get(url)
if response.status_code != 200:
    print("unable to retrieve")
else:
    print("retrieved")

retrieved


In [3]:
soup = BeautifulSoup(response.content, 'html.parser')
soup

<!DOCTYPE html>

<html lang="en-us">
<head>
<!-- Favicon stuff -->
<link href="/apple-touch-icon.png" rel="apple-touch-icon" sizes="180x180"/>
<link href="/favicon-32x32.png" rel="icon" sizes="32x32" type="image/png"/>
<link href="/favicon-16x16.png" rel="icon" sizes="16x16" type="image/png"/>
<link href="/site.webmanifest" rel="manifest"/>
<link color="#5bbad5" href="/safari-pinned-tab.svg" rel="mask-icon"/>
<meta content="#da532c" name="msapplication-TileColor"/>
<meta content="#ffffff" name="theme-color"/>
<meta name="ir-site-verification-token" value="-1714523169">
<link href="https://www.caffeineinformer.com/wp-content/cache/breeze-minification/css/breeze_30b71c605b29e3c385bea87173405798.css" media="all" rel="stylesheet" type="text/css"><title>   Caffeine Content of Drinks</title>
<meta content="Caffeine amounts for over 1,000 drinks including energy drinks, sodas, coffees, and teas. Which drinks have the most caffeine and which pose a safety risk? " name="description"/>
<meta con

In [4]:
import re
script_tag = soup.find('script', string=lambda s: 'var tbldata =' in str(s))

if script_tag:
    match = re.search(r'var tbldata = (\[.*?\]);', script_tag.string, re.DOTALL)
if match:
    tbldata = match.group(1)
print(tbldata)

[["<a href='/caffeine-content/28-energy-drink-black-white'>28 Black Energy Drink</a>","8.46","125","80","9.5","ED"],["<a href='/caffeine-content/3-caffeine-water'>3 Water </a>","16.9","0","50","3.0","W"],["<a href='/caffeine-content/3d-energy-drink'>3D Energy Drink</a>","16","15","200","12.5","ED"],["<a href='/caffeine-content/4-purpose'>4 Purpose Energy Drink</a>","8.46","70","70","8.3","ED"],["<a href='/caffeine-content/4-c-energy-rush'>4C Energy Drink Mix</a>","16.9","15","170","10.1","ED"],["<a href='/caffeine-content/5-hour-energy'>5 Hour Energy</a>","1.93","4","200","103.6","ES"],["<a href='/caffeine-content/5-hour-extra-strength'>5 Hour Energy Extra Strength</a>","1.93","0","230","119.2","ES"],["<a href='/caffeine-content/7-eleven-brewed-coffee'>7 Eleven Brewed Coffee</a>","16","5","280","17.5","C"],["<a href='/caffeine-content/7-eleven-energy-shot'>7-Eleven Energy Shot</a>","2","0","260","130.0","ES"],["<a href='/caffeine-content/7-up'>7-Up</a>","12","140","0","0.0","S"],["<a h

In [5]:
tbldata = eval(tbldata)
tbldata[0]

["<a href='/caffeine-content/28-energy-drink-black-white'>28 Black Energy Drink</a>",
 '8.46',
 '125',
 '80',
 '9.5',
 'ED']

In [6]:
df = pd.DataFrame(tbldata, columns=["drink_name", "volume_floz", "calories", "caffeine_mg", "caffeine_mg/floz", "drink_type"])
df.head()

,drink_name,volume_floz,calories,caffeine_mg,caffeine_mg/floz,drink_type
0,<a href='/caffeine-content/28-energy-drink-bla...,8.46,125,80,9.5,ED
1,<a href='/caffeine-content/3-caffeine-water'>3...,16.9,0,50,3.0,W
2,<a href='/caffeine-content/3d-energy-drink'>3D...,16,15,200,12.5,ED
3,<a href='/caffeine-content/4-purpose'>4 Purpos...,8.46,70,70,8.3,ED
4,<a href='/caffeine-content/4-c-energy-rush'>4C...,16.9,15,170,10.1,ED


In [16]:
df.drink_name = df.drink_name.str.split('>').str[-2].str[:-3]

In [17]:
df.head()

,drink_name,volume_floz,calories,caffeine_mg,caffeine_mg/floz,drink_type
0,28 Black Energy Drink,8.46,125,80,9.5,ED
1,3 Water,16.9,0,50,3.0,W
2,3D Energy Drink,16,15,200,12.5,ED
3,4 Purpose Energy Drink,8.46,70,70,8.3,ED
4,4C Energy Drink Mix,16.9,15,170,10.1,ED


In [19]:
df.drink_type.value_counts()

drink_type
ED    237
C     173
S      91
T      67
ES     37
W      27
Name: count, dtype: int64

In [20]:
df['drink_type'] = df['drink_type'].replace({
    'ED': 'Energy Drink',
    'C': 'Coffee',
    'S': 'Soda',
    'T': 'Tea',
    'ES': 'Energy Shots',
    'W': 'Water'
})
df.head()

,drink_name,volume_floz,calories,caffeine_mg,caffeine_mg/floz,drink_type
0,28 Black Energy Drink,8.46,125,80,9.5,Energy Drink
1,3 Water,16.9,0,50,3.0,Water
2,3D Energy Drink,16,15,200,12.5,Energy Drink
3,4 Purpose Energy Drink,8.46,70,70,8.3,Energy Drink
4,4C Energy Drink Mix,16.9,15,170,10.1,Energy Drink


In [21]:
df.to_csv('drinks_data.csv', index=False)